# Analytical Data Source

# Step 1. Select

Select: Selecting the analysis results for the technical and methodical provision.

Choosing analysis outcomes for technical and systematic planning.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import pandas as pd
import numpy as np
from torch.optim import lr_scheduler
import time
import os
from tempfile import TemporaryDirectory

df = pd.read_csv('../data/ds_salaries_cleaned.csv')

# Step 2. Request

Request: If the analysis results do not suit the intended technical and methodical provision, changes can be requested. 

The analysis results meet expectations, so skip this step.

# Step 3. Select

Select: This process aligns well with the requirement that data selected for technical and methodical provision should be appropriately prepared. If new data are introduced, they should be processed in the same manner to ensure consistency with the original model's data characteristics.

We calculate the 25th, 50th, and 75th percentiles of salaries in the 'salary_in_usd' column of the DataFrame df and define four salary levels based on these percentiles.

In [3]:
# calculate the percentiles
def convertData(data):
    
    percentiles = np.percentile(df['salary_in_usd'], [25, 50, 75])  # 25th, 50th, 75th percentiles
    print(percentiles)
    # define the labels of saraly levels 
    labels = ['Level 1', 'Level 2', 'Level 3', 'Level 4']

    # print the meaning of label 
    for i in range(len(labels)):
        if i == 0:
            print('Level 1: < {:.2f} USD'.format(percentiles[i]))
        elif i == len(labels) - 1:
            print('Level {}: > {:.2f} USD'.format(i+1, percentiles[i-1]))
        else:
            print('Level {}: {:.2f} ~ {:.2f} USD'.format(i+1, percentiles[i-1], percentiles[i]))


    #  -np.inf, np.inf be the lower and upper bounds of the bins
    # *percentiles: base on the percentiles to cut the bins and generate the salary levels for being the labels of training data
    df['salary_level'] = pd.cut(df['salary_in_usd'], bins=[-np.inf, *percentiles, np.inf], labels=labels)
    data = df.drop(['salary_in_usd'], axis=1) # we cannot put the label into the training data
    print("\nThe data of each salary level is balanced.")
    print(data['salary_level'].value_counts())
    return data



# Step 4. Request

Request: If the selected data do not suit the intended technical and methodical provision, changes can be requested. 

The selected data suit the intended technical and methodical provision,so skip this step. 

# Step 5. Check

Check: During the technical and methodical provision, the applicability must be checked by 
the target group for the analysis. 

We check by the target group for the analysis. It check the data is balanced or not and calculate each salary level's data

In [4]:
data = convertData(df)
data
# each number of salary level have approximately the same number of data which occupy 25% of the whole data

[ 95000. 135000. 175000.]
Level 1: < 95000.00 USD
Level 2: 95000.00 ~ 135000.00 USD
Level 3: 135000.00 ~ 175000.00 USD
Level 4: > 175000.00 USD

The data of each salary level is balanced.
salary_level
Level 2    967
Level 1    956
Level 4    932
Level 3    900
Name: count, dtype: int64


,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,salary_level
0,2023,Senior-level/Expert,Full-Time,Data Scientist,ES,100,ES,L,Level 1
1,2023,Mid-level/Intermediate,Contract,ML Engineer,US,100,US,S,Level 1
2,2023,Mid-level/Intermediate,Contract,ML Engineer,US,100,US,S,Level 1
3,2023,Senior-level/Expert,Full-Time,Data Scientist,CA,100,CA,M,Level 3
4,2023,Senior-level/Expert,Full-Time,Data Scientist,CA,100,CA,M,Level 2
...,...,...,...,...,...,...,...,...,...
3750,2020,Senior-level/Expert,Full-Time,Data Scientist,US,100,US,L,Level 4
3751,2021,Mid-level/Intermediate,Full-Time,Data Scientist,US,100,US,L,Level 3
3752,2020,Entry-level/Junior,Full-Time,Data Scientist,US,100,US,S,Level 2
3753,2020,Entry-level/Junior,Contract,Data Analyst,US,100,US,L,Level 2


# Step 6. Apply

Apply: Identified opportunities for ensuring applicability must be considered during the technical and methodical provision.

We create the function to convert the data for training and testing, ensuring applicability by turning the data into tensors and normalizing it, while considering identified opportunities during the technical and methodical provision.

In [9]:
class SalaryDataset(Dataset):

    # Initialization function for loading and preprocessing data
    def __init__(self, data, train=True, transform=None):
        # The 'train' parameter indicates whether the data is for training or testing
        self.train = train

        # Create MinMaxScaler and OneHotEncoder for data preprocessing
        minmax_scaler = MinMaxScaler()
        onehot_enc = OneHotEncoder()

        # Split data into categorical features, numerical features, and labels
        categorical_features = data[data.select_dtypes(include=['object']).columns]
        numerical_features = data[data.select_dtypes(exclude=['object']).columns].drop('salary_level', axis=1)
        label_features = data[['salary_level']]

        # Normalize numerical features (MinMax normalization)
        numerical_features_arr = minmax_scaler.fit_transform(numerical_features)

        # Perform one-hot encoding on categorical features
        categorical_features_arr = onehot_enc.fit_transform(categorical_features).toarray()

        # After one-hot encoding categorical features, retrieve the encoded feature names
        label_features = onehot_enc.fit_transform(label_features).toarray()

        # Combine normalized numerical features and one-hot encoded categorical features into one dataset
        combined_features = pd.DataFrame(data=numerical_features_arr, columns=numerical_features.columns)
        combined_features = pd.concat([combined_features, pd.DataFrame(data=categorical_features_arr)], axis=1)
        combined_features = pd.concat([combined_features, pd.DataFrame(data=label_features)], axis=1).reset_index(drop=True)

        # Split the dataset into training and testing sets
        train_data, test_data = train_test_split(combined_features, test_size=0.2, random_state=42)

        # Choose the data to use based on the training or testing mode
        if train:
            self.data = train_data
        else:
            self.data = test_data

    # Returns the length of the dataset
    def __len__(self):
        return len(self.data)

    # Function for training the neural network, returns features and labels
    def __getitem__(self, idx):
        # Retrieve the data from the ith row in the self.data DataFrame
        sample = self.data.iloc[idx] 
        # Convert a data structure into a PyTorch tensor and specify the tensor's data type as float
        features = torch.FloatTensor(sample[:-4])
        label = torch.FloatTensor(sample[-4:])
        return features, label

    # Returns the entire dataset as a DataFrame
    def getData(self):
        return self.data


# Step 7. Check

Check: During the technical and methodical provision, technical requirements must be examined.

Create«

In [10]:
train_dataset = SalaryDataset(data, train=True) 
test_dataset = SalaryDataset(data, train=False)
print('train_dataset len:', len(train_dataset))
print('val_dataset len:', len(test_dataset))
print('total_dataset len:', len(train_dataset) + len(test_dataset))

train_dataset len: 3004
val_dataset len: 751
total_dataset len: 3755


In [11]:
test_dataset.getData().shape

(751, 178)

In [12]:
data.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,salary_level
0,2023,Senior-level/Expert,Full-Time,Data Scientist,ES,100,ES,L,Level 1
1,2023,Mid-level/Intermediate,Contract,ML Engineer,US,100,US,S,Level 1
2,2023,Mid-level/Intermediate,Contract,ML Engineer,US,100,US,S,Level 1
3,2023,Senior-level/Expert,Full-Time,Data Scientist,CA,100,CA,M,Level 3
4,2023,Senior-level/Expert,Full-Time,Data Scientist,CA,100,CA,M,Level 2


# Step 8. Apply 

Apply: Any technical impossibilities for implementation identified must be considered during the technical and methodical provision.

It involves technical setup for handling data, which is a crucial part of the methodical provision in machine learning or data processing projects. Ensuring the correct handling and sizing of datasets is vital to the success of such projects, and any technical impossibilities or limitations identified in this area must be addressed.

In [13]:
# Create DataLoader for training and testing datasets 
dataloaders = {'train': train_dataset, 'test': test_dataset}
dataset_sizes = {x: len(dataloaders[x]) for x in ['train', 'test']}
dataset_sizes

{'train': 3004, 'test': 751}

The function for training and testing data is created.
This will train 25 epochs, and save the best model during training. 

In [14]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=1):
    since = time.time()
    with TemporaryDirectory() as tempdir:
        # Path to save the best model parameters
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')
        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'test']:
                # Set model to training mode or evaluation mode
                if phase == 'train':
                    model.train()  # Training mode
                else: 
                    model.eval()   # Evaluation mode
                
                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs_vector, labels_vector in dataloaders[phase]:

                    # Zero the parameter gradients
                    optimizer.zero_grad()

                    # Forward pass: compute predicted outputs by passing inputs to the model
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs_vector)

                        # Predictions and actual results
                        pred = torch.argmax(outputs).item()
                        result = torch.argmax(labels_vector).item()
                        # Calculate loss
                        loss = criterion(outputs, labels_vector)

                        # Backward pass and optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # Statistics: accumulate loss and count correct predictions
                    running_loss += loss.item()
                    if pred == result:
                        running_corrects += 1

                # Learning rate scheduler step in training phase
                if phase == 'train':
                    scheduler.step()

                # Calculate loss and accuracy for each epoch
                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects / dataset_sizes[phase]
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} Time elapsed: {round((time.time() - since))} sec.')
                
                # Save the model if it has the best accuracy so far
                if phase == 'test' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        # Print out total time for training and the best accuracy
        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # Load the best model weights before proceeding to the next epoch
        model.load_state_dict(torch.load(best_model_params_path))
    return model


Create a neural network model

In [15]:
class SalaryPredictor(nn.Module):
    def __init__(self, input_size):
        super(SalaryPredictor, self).__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.layer2 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 4)
    
    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.output_layer(x)
        return x


In [16]:
D_in, D_out = 174, 4
model = SalaryPredictor(D_in)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_function = nn.CrossEntropyLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Step 9. Provide

Provide: The results of the technical and methodical provision form the basis for the professional provision of the prepared analysis results for the target user group. The effects can range from a support of existing processes to a project adjustment to a completely new development (now automated, if applicable) of processes.

Technical provision results underpin targeted user analysis with epoch data. And finally save the best prediction result of model, which is easy to deploy and use in the future. 

In [17]:
model = train_model(model, loss_function, optimizer, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 1.2816 Acc: 0.3692 Time elapsed: 4 sec.
test Loss: 1.1817 Acc: 0.4474 Time elapsed: 4 sec.

Epoch 1/24
----------
train Loss: 1.1470 Acc: 0.4534 Time elapsed: 8 sec.
test Loss: 1.1508 Acc: 0.4780 Time elapsed: 8 sec.

Epoch 2/24
----------
train Loss: 1.1243 Acc: 0.4640 Time elapsed: 12 sec.
test Loss: 1.1442 Acc: 0.4820 Time elapsed: 12 sec.

Epoch 3/24
----------
train Loss: 1.1144 Acc: 0.4707 Time elapsed: 16 sec.
test Loss: 1.1387 Acc: 0.4874 Time elapsed: 17 sec.

Epoch 4/24
----------
train Loss: 1.1082 Acc: 0.4734 Time elapsed: 21 sec.
test Loss: 1.1383 Acc: 0.4887 Time elapsed: 21 sec.

Epoch 5/24
----------
train Loss: 1.1030 Acc: 0.4824 Time elapsed: 25 sec.
test Loss: 1.1316 Acc: 0.4900 Time elapsed: 26 sec.

Epoch 6/24
----------
train Loss: 1.0988 Acc: 0.4820 Time elapsed: 29 sec.
test Loss: 1.1288 Acc: 0.4887 Time elapsed: 30 sec.

Epoch 7/24
----------
train Loss: 1.0795 Acc: 0.4887 Time elapsed: 34 sec.
test Loss: 1.1088 Acc: 0.4700 Tim

In [18]:
# save the model to disk 
torch.save(model.state_dict(), '../data/model/salary_pred.pt')

### Predict the value with the model
1. Get all data from the source.
2. Add new data to the source.
3. Convert the data to datasource for model prediction.
4. Get the model and predict the value with the last data in the source. 

> 1. Get all data from the source

In [19]:
df = pd.read_csv('../data/ds_salaries_cleaned.csv')
data = convertData(df)

[ 95000. 135000. 175000.]
Level 1: < 95000.00 USD
Level 2: 95000.00 ~ 135000.00 USD
Level 3: 135000.00 ~ 175000.00 USD
Level 4: > 175000.00 USD

The data of each salary level is balanced.
salary_level
Level 2    967
Level 1    956
Level 4    932
Level 3    900
Name: count, dtype: int64


> 2. Add new data to the source

In [20]:
hight_input = {
    'work_year': 2023,
    'experience_level': 'Senior-level/Expert',
    'employment_type': 'Full-Time',
    'job_title': 'Data Science Manager',
    'employee_residence': ['US'],
    'remote_ratio': [100],
    'company_location': ['US'],
    'company_size': ['L'],
    'salary_level': ['Level 1'] # we don't care about the label
}

low_intput = {
    'work_year': 2023,
    'experience_level': 'Entry-level/Junior',
    'employment_type': 'Full-Time',
    'job_title': 'Data Analyst',
    'employee_residence': ['US'],
    'remote_ratio': [100],
    'company_location': ['US'],
    'company_size': ['L'],
    'salary_level': ['Level 1'] # we don't care about the label
}

input = low_intput

data = pd.concat([data, pd.DataFrame(input)], ignore_index=True)
data.tail()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,salary_level
3751,2021,Mid-level/Intermediate,Full-Time,Data Scientist,US,100,US,L,Level 3
3752,2020,Entry-level/Junior,Full-Time,Data Scientist,US,100,US,S,Level 2
3753,2020,Entry-level/Junior,Contract,Data Analyst,US,100,US,L,Level 2
3754,2021,Senior-level/Expert,Full-Time,Data Science Manager,IN,50,IN,L,Level 1
3755,2023,Entry-level/Junior,Full-Time,Data Analyst,US,100,US,L,Level 1


> 3. Convert the data to datasource for model prediction.

In [21]:
data.dtypes

work_year              int64
experience_level      object
employment_type       object
job_title             object
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
salary_level          object
dtype: object

In [22]:
class DatasetConverter(Dataset):

    # Initialization function for loading and preprocessing data
    def __init__(self, data, transform=None):
        # Create MinMaxScaler and OneHotEncoder for data preprocessing
        minmax_scaler = MinMaxScaler()
        onehot_enc = OneHotEncoder()

        # Split data into categorical features, numerical features, and labels
        categorical_features = data[data.select_dtypes(include=['object']).columns].drop('salary_level', axis=1)
        categorical_columns = data.select_dtypes(include=['object']).columns
        print(f'categorical_features=${categorical_columns}')

        numerical_features = data[data.select_dtypes(exclude=['object']).columns]
        numerical_columns = data.select_dtypes(exclude=['object']).columns
        print(f'numerical_features=${numerical_columns}')
        
        label_features = data[['salary_level']]

        # Normalize numerical features (MinMax normalization)
        numerical_features_arr = minmax_scaler.fit_transform(numerical_features)

        # Perform one-hot encoding on categorical features
        categorical_features_arr = onehot_enc.fit_transform(categorical_features).toarray()

        # After one-hot encoding categorical features, retrieve the encoded feature names
        label_features = onehot_enc.fit_transform(label_features).toarray()

        # Combine normalized numerical features and one-hot encoded categorical features into one dataset
        combined_features = pd.DataFrame(data=numerical_features_arr, columns=numerical_features.columns)
        combined_features = pd.concat([combined_features, pd.DataFrame(data=categorical_features_arr)], axis=1)
        combined_features = pd.concat([combined_features, pd.DataFrame(data=label_features)], axis=1).reset_index(drop=True)

        self.data = combined_features

    # Returns the length of the dataset
    def __len__(self):
        return len(self.data)

    # Function for retrieving features and labels for training the neural network
    def __getitem__(self, idx):
        # Retrieve the data from the ith row in the self.data DataFrame
        sample = self.data.iloc[idx] 
        # Convert a data structure into a PyTorch tensor and specify the tensor's data type as float
        features = torch.FloatTensor(sample[:-4])
        label = torch.FloatTensor(sample[-4:])
        return features, label

    # Returns the entire dataset as a DataFrame
    def getData(self):
        return self.data


In [23]:
tensor_input = DatasetConverter(data).__getitem__(-1)[0]
tensor_label = DatasetConverter(data).__getitem__(-1)[1]
print(f'tensor_input = ${tensor_input}, \ntensor_label = ${tensor_label}')

categorical_features=$Index(['experience_level', 'employment_type', 'job_title',
       'employee_residence', 'company_location', 'company_size',
       'salary_level'],
      dtype='object')
numerical_features=$Index(['work_year', 'remote_ratio'], dtype='object')
categorical_features=$Index(['experience_level', 'employment_type', 'job_title',
       'employee_residence', 'company_location', 'company_size',
       'salary_level'],
      dtype='object')
numerical_features=$Index(['work_year', 'remote_ratio'], dtype='object')
tensor_input = $tensor([1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.

# Step 10. Request

Request: If the professional provision does not lead to the desired results, changes to the technical and methodical provision can be requested.

In this step, we give the mock data to the model and predict the value. If the result is not as expected, we can request changes to the technical and methodical provision.

In [24]:
# load the model from disk
D_in, D_out = 174, 4
pred_model = SalaryPredictor(D_in)
pred_model.load_state_dict(torch.load('../data/model/salary_pred.pt'))

# predict the salary level of the new data
with torch.no_grad():
    # get the last row of the data
    tensor_data = DatasetConverter(data).__getitem__(-1)
    input = tensor_data[0]
    output = pred_model(input)
    print(f'output = ${output}')
    pred = torch.argmax(output).item()
    print('The predicted salary level is:', pred)

categorical_features=$Index(['experience_level', 'employment_type', 'job_title',
       'employee_residence', 'company_location', 'company_size',
       'salary_level'],
      dtype='object')
numerical_features=$Index(['work_year', 'remote_ratio'], dtype='object')
output = $tensor([ 1.7652,  0.6796, -0.7465, -2.0051])
The predicted salary level is: 0
